<a href="https://colab.research.google.com/github/clam004/together/blob/main/together_finetune_law_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is an example before and after demonstration of using Together's finetune API to adapt a base model to a domain specific dataset in law. The dataset consists of legal related questions and answers.

In [1]:
!pip install datasets
!pip install together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 1.6 MB/s eta 0:00:00


In [2]:
import os
import random

from datasets import load_dataset

import together

TOGETHER_API_KEY = "xxx" # replace "xxx" with the string of your together API key

# check to make sure you have the right key, ie: xxxx.....c04c
print(f"using TOGETHER_API_KEY ending in {TOGETHER_API_KEY[-4:]}")

together.api_key = TOGETHER_API_KEY

using TOGETHER_API_KEY ending in c04c


In [3]:
legal_dataset = load_dataset("nisaar/LLAMA2_Legal_Dataset_4.4k_Instructions")
print(legal_dataset)
print("-"*50)
print(legal_dataset['train'][0])
print("-"*50)
print(legal_dataset['train'][1])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'prompt', 'text'],
        num_rows: 4394
    })
})
--------------------------------------------------
{'instruction': 'Analyze and explain the legal reasoning behind the judgment in the given case.', 'input': 'Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278', 'output': "The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck 

In [5]:
def format_to_llama2_chat(system_prompt, user_model_chat_list):

    """ this function follows from
    https://docs.together.ai/docs/fine-tuning-task-specific-sequences
    in roder to format your data into lama-2 chat format

    Args:
    system_prompt

    """

    growing_prompt = f"""<s>[INST] <<SYS>> {system_prompt} <</SYS>>"""

    for user_msg, model_answer in user_model_chat_list:
        growing_prompt += f""" {user_msg} [/INST] {model_answer} </s>"""

    return growing_prompt

format_to_llama2_chat(
    "You are a good robot",
    [("hi robot", "hello human"),("are you good?", "yes im good"),("are you bad?", "no, im good")]
)

'<s>[INST] <<SYS>> You are a good robot <</SYS>> hi robot [/INST] hello human </s> are you good? [/INST] yes im good </s> are you bad? [/INST] no, im good </s>'

In [6]:
data_list = []

for sample in legal_dataset['train']:

    instruction_input_separator = random.choice([":", ": ", "\n", "\n\n", " "])
    input = sample['input'] if sample['input'] is not None else ""
    instruction = sample['instruction'] if sample['instruction'] is not None else ""

    training_sequence = format_to_llama2_chat(
        "you are a helpful legal assistant",
        [(instruction+instruction_input_separator+input,sample['output'])]
    )

    data_list.append({
        "text":training_sequence
    })

print(len(data_list))
print(data_list[0])

4394
{'text': "<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.\n\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST] The Supreme Court in this case applied a broad interpretation of the term 'State' under Article 12 of the Constitution. The court reasoned that a government company undertaking public functions qualifies as 'State' based on factors like government control, public importance of activities etc. This interpretation was based on previous decisions that have defined 'State' under Article 12 broadly to include various agencies and instrumentalities beyond just statutory bodies. The court also applied the principle that unreasonable and arbitrary contractual terms can be struck down under Article 14 of the Constitution. The court found that Rule 9(i) of the service rules, which allowed for termination of service without r

In [7]:
together.Files.save_jsonl(data_list, "legal_dataset.jsonl")

Wrote 4394 records to legal_dataset.jsonl


In [8]:
base_model_name = "togethercomputer/llama-2-7b-chat"
resp = together.Files.check(file="legal_dataset.jsonl", model=base_model_name)
print(resp)

{'is_check_passed': True, 'model_special_tokens': 'the end of sentence token for this model is </s>', 'file_present': 'File found', 'file_size': 'File size 0.007 GB', 'num_samples': 4394, 'num_samples_w_eos_token': 4394}


In [9]:
file_resp = together.Files.upload(file="legal_dataset.jsonl")
file_id = file_resp["id"]
print(file_resp)

Uploading legal_dataset.jsonl: 100%|██████████| 6.96M/6.96M [00:01<00:00, 4.27MB/s]

{'filename': 'legal_dataset.jsonl', 'id': 'file-5d1b910e-2efb-46e1-b482-3321067dda11', 'object': 'file', 'report_dict': {'is_check_passed': True, 'model_special_tokens': 'we are not yet checking end of sentence tokens for this model', 'file_present': 'File found', 'file_size': 'File size 0.007 GB', 'num_samples': 4394, 'num_samples_w_eos_token': 0}}


In [10]:
ft_resp = together.Finetune.create(
  training_file = file_id ,
  model = base_model_name,
  n_epochs = 2,
  batch_size = 4,
  n_checkpoints = 1,
  learning_rate = 1e-4,
  suffix = 'law',
)

fine_tune_id = ft_resp['id']
print(ft_resp)

{'training_file': 'file-5d1b910e-2efb-46e1-b482-3321067dda11', 'model_output_name': 'carson/llama-2-7b-chat-law-2023-09-22-20-50-12', 'model_output_path': 's3://together-dev/finetune/64c4302a5cb247a0c80a3ddb/carson/llama-2-7b-chat-law-2023-09-22-20-50-12/ft-b86b70bf-a49d-4cb2-9125-87cc072ef527', 'Suffix': 'law', 'model': 'togethercomputer/llama-2-7b-chat', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 0.0001, 'user_id': '64c4302a5cb247a0c80a3ddb', 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-09-22T20:50:12.708Z', 'updated_at': '2023-09-22T20:50:12.708Z', 'status': 'pending', 'owner_address': '0xef5286fc0a1ac5bc4d4221cf3d51f1d97c45eaf7', 'id': 'ft-b86b70bf-a49d-4cb2-9125-87cc072ef527', 'job_id': '', 'token_count': 0, 'param_count': 0, 'total_price': 0, 'epochs_completed': 0, 'events': [{'object': 'fine-tune-event', 'created_at': '2023-09-22T20:50:12.708Z', 'level': '', 'message': 'Fine tune request create

In [11]:
print(together.Finetune.retrieve(fine_tune_id=fine_tune_id)) # retrieves information on finetune event
print("-"*50)
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed
print(together.Finetune.is_final_model_available(fine_tune_id=fine_tune_id)) # True, False
print(together.Finetune.get_checkpoints(fine_tune_id=fine_tune_id)) # list of checkpoints

{'training_file': 'file-5d1b910e-2efb-46e1-b482-3321067dda11', 'model_output_name': 'carson/llama-2-7b-chat-law-2023-09-22-20-50-12', 'model_output_path': 's3://together-dev/finetune/64c4302a5cb247a0c80a3ddb/carson/llama-2-7b-chat-law-2023-09-22-20-50-12/ft-b86b70bf-a49d-4cb2-9125-87cc072ef527', 'Suffix': 'law', 'model': 'togethercomputer/llama-2-7b-chat', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 0.0001, 'user_id': '64c4302a5cb247a0c80a3ddb', 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-09-22T20:50:12.708Z', 'updated_at': '2023-09-22T20:50:16.53Z', 'status': 'running', 'owner_address': '0xef5286fc0a1ac5bc4d4221cf3d51f1d97c45eaf7', 'id': 'ft-b86b70bf-a49d-4cb2-9125-87cc072ef527', 'job_id': '1510', 'token_count': 0, 'param_count': 0, 'total_price': 0, 'epochs_completed': 0, 'events': [{'object': 'fine-tune-event', 'created_at': '2023-09-22T20:50:12.708Z', 'level': '', 'message': 'Fine tune request cre

In [13]:
print(together.Finetune.retrieve(fine_tune_id=fine_tune_id)) # retrieves information on finetune event
print("-"*50)
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed
print(together.Finetune.is_final_model_available(fine_tune_id=fine_tune_id)) # True, False
print(together.Finetune.get_checkpoints(fine_tune_id=fine_tune_id)) # list of checkpoints

{'training_file': 'file-5d1b910e-2efb-46e1-b482-3321067dda11', 'model_output_name': 'carson/llama-2-7b-chat-law-2023-09-22-20-50-12', 'model_output_path': 's3://together-dev/finetune/64c4302a5cb247a0c80a3ddb/carson/llama-2-7b-chat-law-2023-09-22-20-50-12/ft-b86b70bf-a49d-4cb2-9125-87cc072ef527', 'Suffix': 'law', 'model': 'togethercomputer/llama-2-7b-chat', 'n_epochs': 2, 'n_checkpoints': 1, 'batch_size': 4, 'learning_rate': 0.0001, 'user_id': '64c4302a5cb247a0c80a3ddb', 'staring_epoch': 0, 'training_offset': 0, 'checkspoint_path': '', 'random_seed': '', 'created_at': '2023-09-22T20:50:12.708Z', 'updated_at': '2023-09-22T20:53:00.763Z', 'status': 'running', 'owner_address': '0xef5286fc0a1ac5bc4d4221cf3d51f1d97c45eaf7', 'id': 'ft-b86b70bf-a49d-4cb2-9125-87cc072ef527', 'job_id': '1510', 'token_count': 1676600, 'param_count': 6738415616, 'total_price': 5000000000, 'epochs_completed': 0, 'events': [{'object': 'fine-tune-event', 'created_at': '2023-09-22T20:50:12.708Z', 'level': '', 'message

In [12]:
test_chat_prompt = "<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.\n\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST]"
test_chat_prompt

'<s>[INST] <<SYS>> you are a helpful legal assistant <</SYS>> Analyze and explain the legal reasoning behind the judgment in the given case.\n\nCentral Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST]'

In [14]:
new_model_name = 'carson/ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf-2023-09-22-13-47-25'

model_list = together.Models.list()

print(f"{len(model_list)} models available")

available_model_names = [model_dict['name'] for model_dict in model_list]

new_model_name in available_model_names

275 models available


True

In [15]:
together.Models.start(new_model_name)

{'success': True,
 'value': 'a13aabcb44b4dbb108de826b129c6de390804e73aabdde8949262e6633ac906f-3ca2aa611c438e84eb74f756a0f07dfce043e6f40698f5004ecbfb7c43c169a5',
 'wasAlreadyEnabled': True}

In [16]:
together.Models.ready(new_model_name)

AttributeError: ignored

In [ ]:
print(new_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  model = new_model_name,
  max_tokens = 256,
  temperature = 0.6,
  top_k = 90,
  top_p = 0.8,
  repetition_penalty = 1.1,
  stop = ['</s>']
)

# print generated text
print(output['prompt'][0]+" -> "+output['output']['choices'][0]['text'])

Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr., 1986 AIR 1571, 1986 SCR (2) 278 [/INST] -> The Supreme Court's decision in Central Inland Water Transport Corporation Ltd. vs Brojo Nath Ganguly & Anr. was based on several key legal principles and reasoning. Firstly, the court held that Article 14 of the Indian Constitution, which guarantees the right to equality, can apply to contractual terms between State entities and private parties if found arbitrary. This means that even though the employees were not government servants under the definition provided under Article 12, their service conditions could still be subject to constitutional scrutiny. Secondly, the court emphasized that Article 14 does not require proof of malice or mala fides for a termination to be deemed arbitrary. Instead, it is enough to show that the termination violates the principle of reasonableness and non-arbitrariness. Thirdly, the court rejected the doctrine of 'State instrumentality' as a valid defense against writ jurisdiction, stating that an entity will be considered 'State' only if there is deep and pervasive governmental control in its functioning and financing. Lastly, the court clarified that the right to livelihood under Article 21 is affected by arbitrary termination of employment. Therefore, ....